In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

import numpy as np

from math import log2

import os

from utils import save_samples, gradient_penalty, save_checkpoint,load_checkpoint

from tqdm import tqdm

In [3]:
IMG_SIZE_AT_START = 64
IMG_SIZE_AT_END = 64

IMAGE_SAVE_FREQ = 5

BASE_DIR = "."

DATASET_DIR = BASE_DIR+"/data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 0.008
IMG_CHANNELS = 3
Z_DIM = 128
LAMBDA_GP = 10

                      #4  8   16 32 64 128 256
BATCH_SIZE_BY_STEP = [0,256,256,256,128,64,48]

                      #4  8 16 32   64 128 256
PROGRESSIVE_EPOCHS = [0,0,0,0      ,200, 0,  0]

MODELS_DIR = BASE_DIR+"/models"
os.makedirs(MODELS_DIR,exist_ok=True)

LOAD_MODEL = True

LOAD_MODEL_DIR = MODELS_DIR+"/64_64_lr_0.008_epoch_[0, 0, 0, 0, 100, 0, 0]/64"

RETRAIN= True

In [4]:
step = int(log2(IMG_SIZE_AT_START/4))
#number of layers to use at start

In [5]:
def get_loader(img_size,step):
    transform =transforms.Compose(
        [
            transforms.Resize((img_size,img_size)),
            #transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(
                [0.5 for _ in range(3)],
                [0.5 for _ in range(3)]
            ),
            
            transforms.RandomHorizontalFlip(p=0.5)
        ]
    )
    #print(int(log2(img_size/4)))
    
    batch_size = BATCH_SIZE_BY_STEP[step]
    
    
    
    dataset = datasets.ImageFolder(root = DATASET_DIR,transform=transform)
    
    loader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = True,
        num_workers = 4,
        pin_memory=True
        
        
    )
    
    return loader, dataset

In [6]:
def train_fn(
    critic,
    gen,
    loader,
    dataset,
    step,
    alpha,
    opt_critic,
    opt_gen,
    scaler_gen,
    scaler_critic,
):
    loop = tqdm(loader, leave=True)
    for batch_idx, (real, _) in enumerate(loop):
        real = real.to(DEVICE)
        cur_batch_size = real.shape[0]

        # Train Critic: max E[critic(real)] - E[critic(fake)] <-> min -E[critic(real)] + E[critic(fake)]
        # which is equivalent to minimizing the negative of the expression
        noise = torch.randn(cur_batch_size, Z_DIM, 1, 1).to(DEVICE)

        with torch.cuda.amp.autocast():
            fake = gen(noise, alpha, step)
            critic_real = critic(real, alpha, step)
            critic_fake = critic(fake.detach(), alpha, step)
            gp = gradient_penalty(critic, real, fake, alpha, step, device=DEVICE)
            loss_critic = (
                -(torch.mean(critic_real) - torch.mean(critic_fake))
                + LAMBDA_GP * gp
                + (0.001 * torch.mean(critic_real ** 2))
            )

        opt_critic.zero_grad()
        scaler_critic.scale(loss_critic).backward()
        scaler_critic.step(opt_critic)
        scaler_critic.update()

        # Train Generator: max E[critic(gen_fake)] <-> min -E[critic(gen_fake)]
        with torch.cuda.amp.autocast():
            gen_fake = critic(fake, alpha, step)
            loss_gen = -torch.mean(gen_fake)

        opt_gen.zero_grad()
        scaler_gen.scale(loss_gen).backward()
        scaler_gen.step(opt_gen)
        scaler_gen.update()

        # Update alpha and ensure less than 1
        alpha += cur_batch_size / (
            (PROGRESSIVE_EPOCHS[step] * 0.5) * len(dataset)
        )
        alpha = min(alpha, 1)


    return alpha

In [7]:
from model import Generator, Discriminator

gen = Generator().to(DEVICE)
crit = Discriminator().to(DEVICE)

opt_gen = optim.Adam(gen.parameters(),lr = LEARNING_RATE,betas = (0.0,0.99))
opt_crit = optim.Adam(crit.parameters(),lr = LEARNING_RATE,betas = (0.0,0.99))

scaler_gen = torch.cuda.amp.GradScaler()
scaler_crit = torch.cuda.amp.GradScaler()

gen.train()
crit.train()

this_model_dir = "{}/{}_{}_lr_{}_epoch_{}".format(MODELS_DIR,IMG_SIZE_AT_START,IMG_SIZE_AT_END,LEARNING_RATE,PROGRESSIVE_EPOCHS)

os.makedirs(this_model_dir,exist_ok=True)

In [8]:


if LOAD_MODEL:
    load_checkpoint(f"{LOAD_MODEL_DIR}/gen.pth.tar",gen,opt_gen,LEARNING_RATE,DEVICE)
    load_checkpoint(f"{LOAD_MODEL_DIR}/crit.pth.tar",crit, opt_crit, LEARNING_RATE,DEVICE)

=> Loading checkpoint
=> Loading checkpoint


In [9]:
image_dir = "{}/images".format(this_model_dir)

os.makedirs(image_dir,exist_ok=True)

noise = torch.randn(64,Z_DIM,1,1).to(DEVICE)

In [10]:
for num_epochs in PROGRESSIVE_EPOCHS[step:]:
    alpha = 0.00005 if not RETRAIN else 1
    loader,dataset = get_loader(4*2**step,step)
    
    for epoch in range(num_epochs):
        print(f"Epoch [{epoch+1}/{num_epochs}], Step: {step}, Alpha: {alpha}")
    
        alpha = train_fn(crit,
                         gen,
                        loader,
                        dataset,
                        step,
                        alpha,
                        opt_crit,
                        opt_gen,
                        scaler_gen,
                        scaler_crit
                                )
        if epoch % IMAGE_SAVE_FREQ == 0:
            img_name = "Size{}_Epoch{}_{}.png".format(4*2**step,epoch+1,num_epochs)
            save_samples(gen,noise,8,os.path.join(image_dir,img_name),alpha,step)
            
            
        if 4*2**step >=256:
            dir_model_cp = "{}/{}".format(this_model_dir,4*2**step)
            os.makedirs(dir_model_cp,exist_ok=True) 

            save_checkpoint(crit,opt_crit,dir_model_cp+"/crit.pth.tar")
            save_checkpoint(gen,opt_gen,dir_model_cp+"/gen.pth.tar")
            
    dir_model_cp = "{}/{}".format(this_model_dir,4*2**step)
    os.makedirs(dir_model_cp,exist_ok=True) 

    save_checkpoint(crit,opt_crit,dir_model_cp+"/crit.pth.tar")
    save_checkpoint(gen,opt_gen,dir_model_cp+"/gen.pth.tar")
    
    
    if 4*2**step >= IMG_SIZE_AT_END:
        img_name = "ENDIMAGE_Size{}_.png".format(4*2**step)
        save_samples(gen,noise,8,os.path.join(image_dir,img_name),alpha,step)
        break
    
    step+=1
    

Epoch [1/200], Step: 4, Alpha: 1


100%|██████████| 7/7 [01:03<00:00,  9.03s/it]


Epoch [2/200], Step: 4, Alpha: 1


100%|██████████| 7/7 [01:00<00:00,  8.71s/it]


Epoch [3/200], Step: 4, Alpha: 1


...

Epoch [199/200], Step: 4, Alpha: 1


100%|██████████| 7/7 [00:46<00:00,  6.61s/it]


Epoch [200/200], Step: 4, Alpha: 1


100%|██████████| 7/7 [00:46<00:00,  6.58s/it]


=> Saving checkpoint
=> Saving checkpoint
